In [4]:
from docx import Document
import json
import glob
import re
from tqdm import tqdm

In [2]:
file_list = glob.glob('../docx/*.docx')
konkordanz = {'01': 'Archivname',
        '02': 'Abteilung',
        '05': 'Straße',
        '06': 'Postfach',
        '07': 'PLZ',
        '08': 'Ort',
        '09': 'Tel./Fax',
        '16': 'Archivstruktur',
        '17': 'Benutzung',
        '18': 'Bemerkungen',
        '20': 'Bestandsgruppen-Signatur',
        '20a-i': 'Bestandsgruppen-Signatur in Sortierform',
        '21': 'Provenienz',
        '22': 'verbale Bestandsbezeichnung',
        '23': 'Zeitangaben zum Gesamtbestand',
        '24': 'Erschließungsmittel',
        '25': 'Bemerkung',
        '26': 'Unterbestand in Zitierform',
        '26a-i': 'Unterbestand in Sortierform',
        '27': 'verbale Benennung des Unterbestands ',
        '28': 'von Bestandsgruppe abweichende Provenienz',
        '29': 'Bemerkungen zum Unterbestand',
        '30': 'Faszikel-Signatur(en)',
        '30a-i': 'Signatur in Sortierform',
        '30z': 'Bestellsignatur',
        '31': 'alte Signatur(en)',
        '32': 'von Signatur abweichender Standort',
        '33': 'Fundstelle im Erschließungsmittel ',
        '34': 'Filmnummer',
        '40': 'Titel',
        '41': 'Enthält-Vermerk',
        '42': 'Beigaben und Begleitmaterial',
        '43': 'Zeitraum',
        '45': 'Bemerkungen zum Faszikel',
        '50': 'Kollationsvermerk',
        '51': 'Formatangabe',
        '52': 'Editionshinweise',
        '55': 'Autopsie, 1=erfolgt, 2=sollte vorgenommen werden; wenn nicht vorhanden Nein'}
db = {'konkordanz': konkordanz, 'archive': {}}
try:
    for file in tqdm(file_list):
        archiv = file[7:-5]
        db['archive'][f'{archiv}'] = {'bestaende': {}}
        doc = Document(file)
        for absatz in doc.paragraphs:
            try:
                absatz_dict = {absatz.text.split("\t")[0].strip():absatz.text.split("\t")[1]}
            except:
                absatz_dict = {'0':'0'}
            key = list(absatz_dict.keys())[0].strip()

            if int(key[:2]) < 20 and int(key[:2]) > 0:
                db['archive'][f'{archiv}'][f'{key}'] = absatz_dict[f'{key}']

            elif key == '20':
                bestand = absatz_dict[f'{key}']
                db['archive'][f'{archiv}']['bestaende'][f'{bestand}'] = absatz_dict

            elif re_key := re.match('20[a-z]',key):
                if len(list(db['archive'][f'{archiv}']['bestaende'].keys())) > 0:#es gibt schon bestaende
                    #letzter bestand wurde mit 20 eröffnet, dann wird die 20a-i als zusätzliche info eingetragen
                    if '20' in list(db['archive'][f'{archiv}']['bestaende'].keys())[-1]:
                        db['archive'][f'{archiv}']['bestaende'][f'{bestand}'][f'{key}'] = absatz_dict[f'{key}']
                    #der letzte wurde nicht mit einer 20 eröffnet, deswegen machen wir jetzt einen neuen mit der 20a auf
                    else:
                        bestand = absatz_dict[f'{key}']
                        db['archive'][f'{archiv}']['bestaende'][f'{bestand}'] = absatz_dict
                else:#es gibt noch keine bestaende, dann wird hier ein neuer mit der 20a-i angelegt
                    bestand = absatz_dict[f'{key}']
                    db['archive'][f'{archiv}']['bestaende'] = {f'{bestand}': absatz_dict}    

            elif (int(key[:2]) > 20 and int(key[:2]) <= 25):
                db['archive'][f'{archiv}']['bestaende'][f'{bestand}'][f'{key}'] = absatz_dict[f'{key}']

            elif key == '26':
                unterbestand = absatz_dict[f'{key}']
                if 'unterbestaende' in db['archive'][f'{archiv}']['bestaende'][f'{bestand}']:
                    db['archive'][f'{archiv}']['bestaende'][f'{bestand}']['unterbestaende'][f'{unterbestand}'] = absatz_dict
                else:
                    db['archive'][f'{archiv}']['bestaende'][f'{bestand}']['unterbestaende'] = {f'{unterbestand}': absatz_dict}

            elif (re_key := re.match('26[a-i]',key)) or (key == '27'):
                if 'unterbestaende' in db['archive'][f'{archiv}']['bestaende'][f'{bestand}']:#es gibt schon unterbestaende
                    #letzter unterbestand wurde mit 26 eröffnet
                    if '26' in list(db['archive'][f'{archiv}']['bestaende'][f'{bestand}']['unterbestaende'][f'{unterbestand}'].keys())[-1]:
                        db['archive'][f'{archiv}']['bestaende'][f'{bestand}']['unterbestaende'][f'{unterbestand}'][f'{key}'] = absatz_dict[f'{key}']
                    #der letzte wurde nicht mit einer 26 eröffnet, deswegen machen wir jetzt einen neuen mit der 26a auf
                    else:
                        unterbestand = absatz_dict[f'{key}']
                        db['archive'][f'{archiv}']['bestaende'][f'{bestand}']['unterbestaende'][f'{unterbestand}'] = absatz_dict
                else:#es gibt noch keine unterbestaende, dann wird hier ein neuer mit der 26a-i angelegt
                    unterbestand = absatz_dict[f'{key}']
                    db['archive'][f'{archiv}']['bestaende'][f'{bestand}']['unterbestaende'] = {f'{unterbestand}': absatz_dict}

            elif int(key[:2]) > 27 and int(key[:2]) <= 29:
                #print(absatz_dict)
                db['archive'][f'{archiv}']['bestaende'][f'{bestand}']['unterbestaende'][f'{unterbestand}'][f'{key}'] = absatz_dict[f'{key}']

            elif key == '30':
                faszikel = absatz_dict[f'{key}']
                if 'unterbestaende' in db['archive'][f'{archiv}']['bestaende'][f'{bestand}']:
                    if 'faszikel' in db['archive'][f'{archiv}']['bestaende'][f'{bestand}']['unterbestaende'][f'{unterbestand}']:
                        db['archive'][f'{archiv}']['bestaende'][f'{bestand}']['unterbestaende'][f'{unterbestand}']['faszikel'][f'{faszikel}'] = absatz_dict
                    else:
                        db['archive'][f'{archiv}']['bestaende'][f'{bestand}']['unterbestaende'][f'{unterbestand}']['faszikel'] = {f'{faszikel}': absatz_dict}
                else:
                    if 'faszikel' in db['archive'][f'{archiv}']['bestaende'][f'{bestand}']:
                        db['archive'][f'{archiv}']['bestaende'][f'{bestand}']['faszikel'][f'{faszikel}'] = absatz_dict
                    else:
                        db['archive'][f'{archiv}']['bestaende'][f'{bestand}']['faszikel'] = {f'{faszikel}': absatz_dict}
                #print(faszikel, absatz_dict)

            elif (int(key[:2]) > 30) or (re_key := re.match('30[a-z]',key)):
                if 'unterbestaende' in db['archive'][f'{archiv}']['bestaende'][f'{bestand}']:
                    db['archive'][f'{archiv}']['bestaende'][f'{bestand}']['unterbestaende'][f'{unterbestand}']['faszikel'][f'{faszikel}'][f'{key}'] = absatz_dict[f'{key}']
                else:
                    db['archive'][f'{archiv}']['bestaende'][f'{bestand}']['faszikel'][f'{faszikel}'][f'{key}'] = absatz_dict[f'{key}']
            elif key == '0':
                None
        if 'faszikel' in globals():
            del faszikel
        if 'unterbestand' in globals():
            del unterbestand
        if 'bestand' in globals():
            del bestand
        if archiv in globals():
            del archiv
except Exception as e:
    print(f'Archiv: {file}, Fehler {e}')
with open(f'../json/2021-03-16-inventar-gesamt.json', 'w') as f:
    json.dump(db, f, indent=2, sort_keys=True, ensure_ascii=False)

100%|██████████| 580/580 [00:41<00:00, 14.03it/s]


In [13]:
meta = {
    'titel': 'Inventar archivalischer Quellen zur Geschichte des deutschen Buchhandels und Verlagswesens im 19. und 20. Jahrhundert',
    'herkunft': 'Deutsches Buch- und Schriftmuseum der Deutschen Nationalbibliothek',
    'projekt': 'Das Projekt "Inventar archivalischer Quellen zur Geschichte des deutschen Buchhandels und Verlagswesens im 19. und 20. Jahrhundert" wurde innerhalb des Programms "Erschließung buch- und bibliotheksgeschichtlicher Quellenbestände" von der Deutschen Forschungsgemeinschaft gefördert. Der Börsenverein des Deutschen Buchhandels e. V. beteiligte sich durch Übernahme von Reisekosten der Projektbereiche Nord und Süd an der Finanzierung Die online-Verfügbarkeit wurde gleichfalls von der Deutschen Forschungsgemeinschaft im Programm "Überregionale Erschließungsvorhaben der Archive" gefördert.',
    'zielgruppe': 'Das Inventar soll in erster Linie der buchgeschichtlichen Forschung einen Überblick über die Quellenlage für allgemeine und spezielle Untersuchungen zur Verlags- und Buchhandelsgeschichte des 19. und 20. Jahrhunderts verschaffen. Das Inventar richtet sich an Archivbenutzer*innen mit Grundkenntnissen des Archivwesens und archivischer Erschließung',
    'gegenstand': 'Ausgewertet wurden alle erreichbaren archivalischen Quellen zur Buchhandels- und Verlagsgeschichte im umfassenden Sinn mit allen sachlichen Querverbindungen zur Kultur-, Wirtschafts-, Rechts- und Literaturgeschichte. Auch Randgebiete wie Leihbibliotheken, Lesezirkel, Zensur sowie der Themenbereich Presse wurden in das Erschließungskonzept aufgenommen. Nicht berücksichtigt wurden die Bereiche Druckindustrie und Bibliothekswesen.',
    'datenerfassung': '1989-1998',
    'neukonvertierung': 'Übertragung der Rohdaten nach JSON, Deutsches Buch- und Schriftmuseum 2021, Dr. André Wendler'
}
db['meta'] = meta

In [5]:
with open('../json/2021-03-08-inventar_gesamt.json', 'r') as f:
    db = json.load(f)

# Auswertung der Daten

In [ ]:
anzahl_archive = 0
anzahl_bestaende = 0
anzahl_unterbestaende = 0
anzahl_faszikel = 0

for archiv in db['archive']:
    anzahl_archive += 1
    #print(archiv)
    for bestand in db['archive'][f'{archiv}']['bestaende']:
        anzahl_bestaende += 1
        #print('   ', bestand)
        if 'unterbestaende' in db['archive'][f'{archiv}']['bestaende'][f'{bestand}']:
            for unterbestand in db['archive'][f'{archiv}']['bestaende'][f'{bestand}']['unterbestaende']:
                anzahl_unterbestaende += 1
                #print('       ', unterbestand)
                if 'faszikel' in db['archive'][f'{archiv}']['bestaende'][f'{bestand}']['unterbestaende'][f'{unterbestand}']:
                    for akte in db['archive'][f'{archiv}']['bestaende'][f'{bestand}']['unterbestaende'][f'{unterbestand}']['faszikel']:
                        anzahl_faszikel += 1
                        #print('           ', akte)
                else:
                    print(f'{archiv} / {bestand} / {unterbestand}')
        elif 'faszikel' in db['archive'][f'{archiv}']['bestaende'][f'{bestand}']:
            for akte in db['archive'][f'{archiv}']['bestaende'][f'{bestand}']['faszikel']:
                anzahl_faszikel += 1
print(f'Archive: {anzahl_archive}')
print(f'Bestände: {anzahl_bestaende}')
print(f'Unterbestände: {anzahl_unterbestaende}')
print(f'Faszikel: {anzahl_faszikel}')